In [1]:
import numpy as np
from tensorflow import keras
import json
import pickle
import random
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
intents = json.loads(open('intents.json').read())
words = []
classes = []
documents = []
ignore_letters = ['?','!','.', ',']

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
print(documents)

[(['safda'], 'a'), (['dafdafas'], 'a'), (['aaa'], 'a'), (['asfda'], 'a'), (['Hi'], 'greeting'), (['khoe', 'khong', 'em', 'oi'], 'greeting'), (['ban', 'khoe', 'khong'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Whats', 'up'], 'greeting'), (['xin', 'chao'], 'greeting'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['tam', 'biet'], 'goodbye'), (['how', 'old'], 'age'), (['how', 'old', 'is', 'tim'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['how', 'old', 'are', 'you'], 'age'), (['age', '?'], 'age'), (['tuoi'], 'age'), (['tuoi', 'cua', 'toi'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['whats', 'your', 'name', '?'], 'name'), (['ten'], 'name'), (['ban', 'ten', 'gi'], 'name'), (['Id', 'like', 'to', 'buy', 'something'], 'shop'), (['whats', 'on', 'the', 'menu

In [5]:
#lemmatize và hạ thấp từng từ và loại bỏ các từ trùng lặp
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
# sắp xếp class
words = sorted(set(words))
classes = sorted(set(classes))
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))
#tạo training data
training =[]
#tạo mảng trống cho đầu ra
output_empty = [0]*len(classes)
#training set, bag of words for every sentence
for document in documents:
    #khởi tạo bag of words:
    bag=[]
    #danh sách các từ được mã hóa cho patterns
    word_patterns = document[0]
    #lemmatize từng từ - tạo từ cơ bản, cố gắng biểu diễn các từ có liên quan
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    #đầu ra là '0' cho mỗi tag và 1 cho tag hiện tại (cho mỗi pattern)
    output_row = list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])

In [6]:
#xáo cái training
random.shuffle(training)
dtype=object
training = np.array(training)
#create training and testting list. X- patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])

C:\Users\ASUS PC\AppData\Local\Temp\ipykernel_13644\217492749.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [7]:
model=keras.Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer= sgd, metrics= ['accuracy'])
hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5',hist)
print("training is done")

Epoch 1/200
10/10 [==============================] - 2s 4ms/step - loss: 2.3567 - accuracy: 0.1042
Epoch 2/200
10/10 [==============================] - 0s 4ms/step - loss: 2.2937 - accuracy: 0.2083
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2460 - accuracy: 0.1458
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1756 - accuracy: 0.1875
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1661 - accuracy: 0.1875
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1182 - accuracy: 0.2917
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0834 - accuracy: 0.3542
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9672 - accuracy: 0.4583
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0238 - accuracy: 0.2917
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8801 - accuracy: 0.3750
Epoch 11/

10/10 [==============================] - 0s 2ms/step - loss: 0.1702 - accuracy: 0.9167
Epoch 84/200
10/10 [==============================] - 0s 1ms/step - loss: 0.2205 - accuracy: 0.8958
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1259 - accuracy: 0.9583
Epoch 86/200
10/10 [==============================] - 0s 1ms/step - loss: 0.2332 - accuracy: 0.9167
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1702 - accuracy: 0.9583
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.2119 - accuracy: 0.8958
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1028 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1512 - accuracy: 0.9583
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.2609 - accuracy: 0.9167
Epoch 92/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1383 - accuracy: 0.9792
Epoch 93/200
